In [74]:
import re
from collections import Counter

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())
Words=Counter(words(open('typo-0.2.txt').read()))


In [91]:
# reading the file with the correct spelling words
text='voc-1bwc.txt'

#declaring a dictionary with the correct words and their frequency as key
word_frequency={}

with open(text,'r') as file:
    for line in file:
        parts=line.strip().split()
        if len(parts)==2:
            word=parts[1]
            frequency=int(parts[0])
            word_frequency[word]=frequency

#adding the words orig to the dictionary 
word_frequency['orig']=Words['orig']


#adding the words in the file to the dictionary if they are not already in it
def WordFrequency(word_to_find):
    if word_to_find in word_frequency:
        return word_frequency[word_to_find]
    else:
        return 0
print(WordFrequency("the"))        


41640816


In [75]:
!pip install python-Levenshtein


Defaulting to user installation because normal site-packages is not writeable


 Levenshtein Distance approach

In [76]:
import Levenshtein

In [ ]:
#Levensthien distance
def distance_levenstein(a, b):
    return Levenshtein.distance(a, b)

In [78]:
import heapq #to add more than word to the correction candidates

In [82]:
# get the closest words (correction) to the word we want to correct using heapq
def get_closest_word(word, words, n):
    closest_words = heapq.nsmallest(n, words, key=lambda x: distance_levenstein(word, x))
    closest_words.sort(key=lambda tup: tup[1])
    return closest_words

In [ ]:
# get the closest words (correction) to the word we want to correct using vanilla python
def get_closest_word(word, words, n):
    closest_words = []
    
    for candidate in words:
        dist = distance_levenstein(word, candidate)
        if len(closest_words) < n:
            closest_words.append((candidate, dist))
        else:
            # Replace the word with the highest distance if the current distance is smaller
            max_dist_word, max_dist = max(closest_words, key=lambda x: x[1])
            if dist < max_dist:
                closest_words.remove((max_dist_word, max_dist))
                closest_words.append((candidate, dist))
    
    closest_words.sort(key=lambda x: x[1])  # Sort by distance
    return [word for word, _ in closest_words]

In [92]:
#find the typos in a text and their correction
def find_typos(text,n):
    typos=[]
    for word in text:
        word=word.lower()
        if WordFrequency(word)==0:
            closest_word=get_closest_word(word, word_frequency.keys(),n)
            typos.append((word, closest_word))
    return typos

In [51]:
import json

In [110]:
# open typo-0.2.txt and read it
text=open('typo-0.2.txt','r').read()


In [187]:
def replace_typo(match):
    orig=match.group(1)
    typo=match.group(2)
    correction_arr=get_closest_word(typo, word_frequency.keys(),5)
    correction = " ".join(correction_arr)
    return f'<correction typo orig="{orig}" >{correction}</correction>'
    

In [188]:
text=open('typo-0.2.txt','r').read()
pattern = r'<typo orig="(.*?)">(.*?)</typo>'
modified_text = re.sub(pattern, replace_typo, text)

# Write the modified content back to the file
with open('typocorrected.txt', 'w') as file:
    file.write(modified_text)

Combination de la correction et de l'insertion dans le texte


In [189]:
#function to mdodify the text using a distance:
def replace_typo_evolved(match, distance_function,n): #n is the number of closest words we want to find
    orig=match.group(1)
    typo=match.group(2)
    correction_arr=distance_function(typo, word_frequency.keys(),n)
    correction = " ".join(correction_arr)
    return f'<correction typo orig="{orig}" >{correction}</correction>'


'Q. Doesn \'t this just hit the very <correction typo orig="wealthy" >wealthy wealth healthy health- realogy</correction> who can <correction typo orig="afford" >afore axford acord alford afford</correction> it ?\nAlso , <correction typo orig="Catholic" >pathologic catholic-muslim anglo-catholic paleolithic anti-catholic</correction> school girl outfits .\nHe wrestled his <correction typo orig="cousins" >cousins cousin mousing clausing corsini</correction> wearing funny <correction typo orig="masks" >masks unmasks maks maass basks</correction> worn by the locals .\nChrysler is in the process of seeking approval from a U.S. Bankruptcy Court judge to sell itself <correction typo orig="to" >tos tog yos tus rtos</correction> a new entity jointly owned <correction typo orig="by" >b bq zb bz vb</correction> Fiat , the United Auto <correction typo orig="Workers" >pokers jokers roker rikers rowers</correction> and <correction typo orig="the" >tthe trhe athe tothe othe</correction> U.S. and Can

Deamarau-Levenstein Distance

In [1]:
!pip install textdistance

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import textdistance #to calculate the Demarau-Levenstein distance

In [3]:
def distance_demarau_levenshtein(a, b):
    return textdistance.damerau_levenshtein(a, b)

In [4]:
# get the closest words (correction) to the word we want to correct using heapq
def get_closest_word_evolved(word, words, distance_function, n):
    closest_words = heapq.nsmallest(n, words, key=lambda x: distance_function(word, x))
    closest_words.sort(key=lambda tup: tup[1])
    return closest_words